In [ ]:
from postprocessing import *
import glob
import tqdm
import os
from natsort import natsorted

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
density = 20
mach = 4
jetpower = 40
start = '15'

In [ ]:
simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
def calculate_ram_pressure(filename):
    snap_data = h5py.File(output_directory + filename, "r")
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]

    center = 0.5 * snap_data["Header"].attrs["BoxSize"]
    v_total_sq = vx ** 2 + vy ** 2 + vz ** 2

    ram_pressure = density * v_total_sq * unit_density * unit_velocity * unit_velocity
    return ram_pressure
    
    
def calculate_thermal_pressure(filename):
    snap_data = h5py.File(output_directory + filename, "r")
    density = snap_data['PartType0/Density'][:]
    temperature = get_temp(output_directory + filename, 5/3)
    number_density = density * rho_to_numdensity 
    thermal_pressure = temperature * number_density * BOLTZMANN

    return thermal_pressure   
    

In [ ]:
def calculate_ratio(filename):
    snap_data = h5py.File(output_directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    temperature = get_temp(output_directory + filename, 5/3)

    center = 0.5 * snap_data["Header"].attrs["BoxSize"]
    v_radial_sq = ((vx * (x - center) + vy * (y - center) + vz * (z - center)) / np.sqrt((x - center) ** 2 +(y - center) ** 2 + (z - center) ** 2)) ** 2
    v_total_sq = vx ** 2 + vy ** 2 + vz ** 2
    number_density = density * rho_to_numdensity 
    
    #ram_pressure = density * v_radial_sq * unit_density * unit_velocity * unit_velocity
    ram_pressure = density * v_total_sq * unit_density * unit_velocity * unit_velocity
    thermal_pressure = temperature * number_density * BOLTZMANN
    
    ratio = (ram_pressure - thermal_pressure) / (ram_pressure + thermal_pressure)
    return ratio

In [ ]:
i_file = 45
filename = "snap_%03d.hdf5" % (i_file)

In [ ]:
ratio = calculate_ratio(filename)

## plotting

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
plot_random(ax, output_directory + filename, value=ratio, limits=[-1, 1], log=False,
            value_name='$(P_{ram} - P_{thermal}) / (P_{ram} + P_{thermal})$', fac=0.5, t0=0)
plt.xticks([500, 700, 900, 1100, 1300, 1500], [0, 200, 400, 600, 800, 1000])
plt.yticks([500, 700, 900, 1100, 1300, 1500], [0, 200, 400, 600, 800, 1000])

In [ ]:
i_file = 8
while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    fig, ax = plt.subplots(figsize=(6,7))
    ratio = calculate_ratio(filename)
    plot_random(ax, output_directory + filename, value=ratio, limits=[-1, 1],
            value_name='$(P_{ram} - P_{thermal}) / (P_{ram} + P_{thermal})$', fac=0.5, t0=0)
    plt.savefig(figures_directory + f'pressure_ratio_total_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/pressure_ratio_total*.png'
ofilename = figures_directory + '/pressure_ratio_total-jet.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=4

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)

## histograms

In [ ]:
i_file = 60
filename = "snap_%03d.hdf5" % (i_file)

In [ ]:
snap_data = h5py.File(output_directory + filename, "r")
x, y, z = snap_data['PartType0/Coordinates'][:].T
vx, vy, vz = snap_data['PartType0/Velocities'][:].T
density = snap_data['PartType0/Density'][:]
masses  = snap_data['PartType0/Masses'][:]
jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
temperature = get_temp(output_directory + filename, 5/3)

center = 0.5 * snap_data["Header"].attrs["BoxSize"]
v_radial_sq = ((vx * (x - center) + vy * (y - center) + vz * (z - center)) / np.sqrt((x - center) ** 2 +(y - center) ** 2 + (z - center) ** 2)) ** 2
v_total_sq = vx ** 2 + vy ** 2 + vz ** 2
number_density = density * rho_to_numdensity 

#ram_pressure = density * v_radial_sq * unit_density * unit_velocity * unit_velocity
ram_pressure = density * v_total_sq * unit_density * unit_velocity * unit_velocity
thermal_pressure = calculate_thermal_pressure(filename)

In [ ]:
mask_jet_left = (jet_tracer > 1e-3) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x < center)
mask_jet_right = (jet_tracer > 1e-3) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x > center)
mask_medium = (jet_tracer < 1e-3) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2 )
mask_shell_left = (jet_tracer < 1e-3) & (jet_tracer > 1e-4) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x < center)
mask_shell_right = (jet_tracer < 1e-3) & (jet_tracer > 1e-4) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x > center)


In [ ]:
bins = np.logspace(-14, -5, 61)
bins_step = bins[1] - bins[0]

In [ ]:
fig, ax = plt.subplots(figsize=(5.5, 3))
ax.hist((ram_pressure[mask_shell_left]), bins=bins,
    log=True, weights=masses[mask_shell_left], label=r'$P_\mathrm{ram}$ jet neighborhood left', alpha=1, lw=2, color='royalblue', histtype='step', linestyle=(0, (5, 1)))
ax.hist((ram_pressure[mask_shell_right]), bins=bins,
    log=True, weights=masses[mask_shell_right], label=r'$P_\mathrm{ram}$ jet neighborhood right', alpha=1, lw=2, color='darkblue', histtype='step')

ax.hist((ram_pressure[mask_jet_left]), bins=bins,
    log=True, weights=masses[mask_jet_left], label=r'$P_\mathrm{ram}$, jet left', alpha=1, lw=2, color='goldenrod', histtype='step', linestyle=(0, (5, 1)))
ax.hist((ram_pressure[mask_jet_right]), bins=bins,
    log=True, weights=masses[mask_jet_right], label=r'$P_\mathrm{ram}$ jet right', alpha=1, lw=2, color='darkgoldenrod', histtype='step')

ax.hist((thermal_pressure[mask_jet_left]), bins=bins,
    log=True, weights=masses[mask_jet_left], label=r'$P_\mathrm{thermal}$, jet left', alpha=1, lw=2, color='lightcoral', histtype='step', linestyle=(0, (5, 1)))
ax.hist((thermal_pressure[mask_jet_right]), bins=bins,
    log=True, weights=masses[mask_jet_right], label=r'$P_\mathrm{thermal}$, jet right', alpha=1, lw=2, color='firebrick', histtype='step')

ax.set_xscale('log')

plt.title(fr't = {np.round(get_time_from_snap(snap_data) * unit_time_in_megayr, 2)} Myr')
plt.legend(loc='upper left',fontsize=7, fancybox=True, framealpha=0.5)
plt.grid(ls='--', zorder = 100, alpha=0.5)

ax.set_xlabel('pressure, Bar')
ax.set_ylabel(r'Mass, $M_\odot$')

ax.set_ylim(1e-1, 1e8)

In [ ]:
i_file = 12 - 1

bins = np.logspace(-14, -5, 61)
bins_step = bins[1] - bins[0]

while True:
    i_file += 1

    try:
        filename = "snap_%03d.hdf5" % (i_file)
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break

    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    masses  = snap_data['PartType0/Masses'][:]
    jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
    temperature = get_temp(output_directory + filename, 5/3)
    
    center = 0.5 * snap_data["Header"].attrs["BoxSize"]
    v_radial_sq = ((vx * (x - center) + vy * (y - center) + vz * (z - center)) / np.sqrt((x - center) ** 2 +(y - center) ** 2 + (z - center) ** 2)) ** 2
    v_total_sq = vx ** 2 + vy ** 2 + vz ** 2
    number_density = density * rho_to_numdensity 
    
    #ram_pressure = density * v_radial_sq * unit_density * unit_velocity * unit_velocity
    ram_pressure = density * v_total_sq * unit_density * unit_velocity * unit_velocity
    thermal_pressure = calculate_thermal_pressure(filename)
    
    mask_jet_left = (jet_tracer > 1e-3) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x < center)
    mask_jet_right = (jet_tracer > 1e-3) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x > center)
    mask_medium = (jet_tracer < 1e-3) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2 )
    mask_shell_left = (jet_tracer < 1e-3) & (jet_tracer > 1e-4) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x < center)
    mask_shell_right = (jet_tracer < 1e-3) & (jet_tracer > 1e-4) & ((x-center) ** 2 + (y-center) ** 2 + (z-center) ** 2 < 500 ** 2) & (x > center)

    fig, ax = plt.subplots(figsize=(5.5, 3))
#     ax.hist((ram_pressure[mask_shell]), bins=bins,
#         log=True, weights=masses[mask_shell], label='jet neighborhood', alpha=0.5, color='tab:blue')
#     ax.hist((ram_pressure[mask_jet]), bins=bins,
#         log=True, weights=masses[mask_jet], label='jet', alpha=0.5, color='tab:red')
    
    ax.hist((ram_pressure[mask_shell_left]), bins=bins,
        log=True, weights=masses[mask_shell_left], label=r'$P_\mathrm{ram}$ jet neighborhood left', alpha=1, lw=2, color='royalblue', histtype='step', linestyle=(0, (5, 1)))
    ax.hist((ram_pressure[mask_shell_right]), bins=bins,
        log=True, weights=masses[mask_shell_right], label=r'$P_\mathrm{ram}$ jet neighborhood right', alpha=1, lw=2, color='darkblue', histtype='step')

    ax.hist((ram_pressure[mask_jet_left]), bins=bins,
        log=True, weights=masses[mask_jet_left], label=r'$P_\mathrm{ram}$, jet left', alpha=1, lw=2, color='goldenrod', histtype='step', linestyle=(0, (5, 1)))
    ax.hist((ram_pressure[mask_jet_right]), bins=bins,
        log=True, weights=masses[mask_jet_right], label=r'$P_\mathrm{ram}$ jet right', alpha=1, lw=2, color='darkgoldenrod', histtype='step')

    ax.hist((thermal_pressure[mask_jet_left]), bins=bins,
                 log=True, weights=masses[mask_jet_left], label=r'$P_\mathrm{thermal}$, jet left', alpha=1, lw=2, color='lightcoral', histtype='step', linestyle=(0, (5, 1)))
    ax.hist((thermal_pressure[mask_jet_right]), bins=bins,
        log=True, weights=masses[mask_jet_right], label=r'$P_\mathrm{thermal}$, jet right', alpha=1, lw=2, color='firebrick', histtype='step')

    ax.set_xscale('log')

    ax.set_xlabel('ram pressure, Ba')
    ax.set_ylabel(r'Mass, $M_\odot$')

    ax.set_ylim(1e-1, 1e8)
    
    plt.title(fr't = {np.round(get_time_from_snap(snap_data) * unit_time_in_megayr, 2)} Myr')
    plt.legend(loc='upper left',fontsize=7, fancybox=True, framealpha=0.5)
    plt.grid(ls='--', zorder = 100, alpha=0.5)
    
    plt.savefig(figures_directory + f'pressure_distribution_{i_file}.png', dpi=400, bbox_inches='tight')
    plt.close()
    

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + f'/pressure_distribution*.png'
ofilename = figures_directory + f'/pressure_distribution.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=2

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)

## pressure as function of time

In [ ]:
i_file = 6
pressures = []

In [ ]:
while True:
    i_file += 1

    try:
        filename = "snap_%03d.hdf5" % (i_file)
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break

    snap_data = h5py.File(output_directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
    temperature = get_temp(output_directory + filename, 5/3)

    center = 0.5 * snap_data["Header"].attrs["BoxSize"]
    number_density = density * rho_to_numdensity 

    thermal_pressure = temperature * number_density * BOLTZMANN
    pressure = np.mean(thermal_pressure[jet_tracer < 1e-3])
    pressures.append([np.round(get_time_from_snap(snap_data) * unit_time_in_megayr, 2), pressure])

In [ ]:
pressures = np.array(pressures).T

In [ ]:
plt.plot(pressures[0], pressures[1])
plt.ylabel('pressure, Bar')
plt.xlabel('time, Myr')

## highlighting high ram pressure

In [ ]:
i_file = 20
filename = "snap_%03d.hdf5" % (i_file)

In [ ]:
snap_data = h5py.File(output_directory + filename, "r")
x, y, z = snap_data['PartType0/Coordinates'][:].T
vx, vy, vz = snap_data['PartType0/Velocities'][:].T
density = snap_data['PartType0/Density'][:]
masses  = snap_data['PartType0/Masses'][:]
jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
temperature = get_temp(output_directory + filename, 5/3)

center = 0.5 * snap_data["Header"].attrs["BoxSize"]
v_radial_sq = ((vx * (x - center) + vy * (y - center) + vz * (z - center)) / np.sqrt((x - center) ** 2 +(y - center) ** 2 + (z - center) ** 2)) ** 2
v_total_sq = vx ** 2 + vy ** 2 + vz ** 2
number_density = density * rho_to_numdensity 

#ram_pressure = density * v_radial_sq * unit_density * unit_velocity * unit_velocity
ram_pressure = density * v_total_sq * unit_density * unit_velocity * unit_velocity

In [ ]:
jet_pressure = ram_pressure[jet_tracer > 1e-3].max()

In [ ]:
high_pressure_zones = np.zeros_like(ram_pressure)
high_pressure_zones_mask = (ram_pressure > jet_pressure)

In [ ]:
high_pressure_zones[high_pressure_zones_mask] = ram_pressure[high_pressure_zones_mask]

In [ ]:
fig, ax = plt.subplots(figsize=(6,7))
# plot_random(ax, output_directory + filename, value=high_pressure_zones, limits=[1e-8, 3e-7], cmap='plasma',
#             value_name='$P_{ram}$, Bar', fac=0.5, t0=15)
plot_random(ax, output_directory + filename, value=abs(calculate_ram_pressure("snap_%03d.hdf5" % (i_file))), log=True, limits=[1e-8, 3e-7], cmap='plasma',
            value_name='$P_{ram}$, Bar', fac=0.5, t0=15)
ax.locator_params(axis='y',nbins=5)
plot_jet_tracer_on_top(ax, output_directory + filename, fac=0.5, t0=15)

In [ ]:
# n0_jet = 14
# i_file = n0_jet # skip snap 0
# while True:
#     i_file += 1
#     filename = "snap_%03d.hdf5" % (i_file)
#     try:
#         snap_data = h5py.File(output_directory + filename, "r")
#     except:
#         break
        
#     fig, ax = plt.subplots(2, 2, figsize=(8, 6.1))
#     fig.tight_layout(w_pad=7.0, h_pad=3.0)
#     plot_dens_vel(ax[0][0], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
#     plot_temp_vel(ax[0][1], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
#     plot_jet_tracer(ax[1][1], output_directory + "snap_%03d.hdf5" % (i_file), fac=0.5, t0=0)
#     plot_random(ax[1][0], output_directory + "snap_%03d.hdf5" % (i_file), value=calculate_ram_pressure("snap_%03d.hdf5" % (i_file)), limits=[1e-8, 3e-7], cmap='plasma',
#             value_name='$P_{ram}$, Bar', fac=0.5, t0=0)
    
    
#     plt.savefig(figures_directory + f'4panels_{i_file}.png', dpi=300, bbox_inches='tight')
#     plt.close()

In [ ]:
# from PIL import Image

# # make gif
# #--------------------------
# def crop_img(im):a
#     width, height = im.size
#     left = 9
#     top =  3
#     right = width - 3
#     bottom = height - 9
#     im = im.crop((left, top, right, bottom))
#     return im

# ifilename = figures_directory + '/4panels*.png'
# ofilename = figures_directory + '/4panels-jet.gif'
# imgs = natsorted(glob.glob(ifilename))

# timestep=4

# frames = []
# for i in imgs:
#     new_frame = Image.open(i)
#     frames.append(crop_img(new_frame))

# frames[0].save(ofilename, format='GIF',
#                append_images=frames[1:],
#                save_all=True,
#                duration=len(imgs) * timestep, loop=0)

In [ ]:
i_file = 12 # skip snap 0
while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
        
    fig, ax = plt.subplots(figsize=(4, 4))
    plot_random(ax, output_directory + filename, value=abs(calculate_ram_pressure("snap_%03d.hdf5" % (i_file))), log=True, limits=[2e-8, 3e-7], cmap='plasma',
                value_name='$P_{ram}$, Bar', fac=0.5, t0=15)
    plot_jet_tracer_on_top(ax, output_directory + filename, fac=0.5, t0=15)
    plt.savefig(figures_directory + f'ram_pressure_slice_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/ram_pressure_slice*.png'
ofilename = figures_directory + '/ram_pressure_slice-jet.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=4

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)

## along x axis

In [ ]:
density = 20
mach = 12
jetpower = 40
start = '15'

In [ ]:
simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
i_file = 12 # skip snap 0
while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    ram_pressure = calculate_ram_pressure(filename)
    jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
    center = 0.5 * snap_data["Header"].attrs["BoxSize"]

    x_axis = np.linspace(0, 500, 101)
    ram_pressure_axis = np.zeros_like(x_axis)
    jet_tracer_axis = np.zeros_like(x_axis)

    for i in range(len(x_axis) -1):
        dist = (x - center - x_axis[i]) ** 2 + (y - center) ** 2 + (z - center) ** 2
        index = np.argmin(dist)
        ram_pressure_axis[i] = ram_pressure[index]
        jet_tracer_axis[i] = jet_tracer[index]

    fig, ax1 = plt.subplots(figsize=(6,4))

    color = 'tab:red'
    ax1.set_xlabel('radius (pc)')
    ax1.set_ylabel('jet tracer', color=color)
    ax1.plot(x_axis, jet_tracer_axis, lw=2, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_ylim(-1e-2, 1.1)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('ram pressure, Bar', color=color)  # we already handled the x-label with ax1
    ax2.plot(x_axis, ram_pressure_axis, lw=2, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim(1e-10, 1e-7)
    ax2.set_yscale('log')
    ax2.grid(ls='--', axis='both', zorder=0)
    fig.tight_layout()

    plt.title("t=%.2f Myr"%(get_time_from_snap(snap_data) * unit_time_in_megayr - 15))
    plt.savefig(figures_directory + f'ram_pressure_jet_tracer_x_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/ram_pressure_jet_tracer_x*.png'
ofilename = figures_directory + '/ram_pressure_jet_tracer_x.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=4

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)

In [ ]:
from multiprocessing import Pool

In [ ]:
def x_loop(i):
    dist = (x - center - x_axis[i]) ** 2 + (y - center) ** 2 + (z - center) ** 2
    index = (dist < 15) & (x < x_axis[i] - dx) & (x > x_axis[i] + dx)  
    
    if (index).sum() > 0:
        return np.mean(ram_pressure[index]), np.mean(jet_tracer[index])
    else:
        index = np.argmin(dist)    
        return ram_pressure[index], jet_tracer[index]

In [ ]:
!ls {output_directory}

In [ ]:
i_file = 13 # skip snap 0
ram_pressure_list_parallel = []
jet_tracer_fall_x_parallel = []
time_parallel = []
# x_axis = np.linspace(-30, -500, 51)
x_axis = np.linspace(30, 500, 51)
dx = x_axis[1] - x_axis[0]

#for i_file in tqdm.trange(12, 42):
    
for i_file in tqdm.trange(13, 78):
    filename = "snap_%03d.hdf5" % (i_file)

    snap_data = h5py.File(output_directory + filename, "r")
    
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data['PartType0/Velocities'][:].T
    density = snap_data['PartType0/Density'][:]
    ram_pressure = density # calculate_ram_pressure(filename) #
    jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
    center = 0.5 * snap_data["Header"].attrs["BoxSize"]

    
#     ram_pressure_axis = np.zeros_like(x_axis)
#     jet_tracer_axis = np.zeros_like(x_axis)
    
    pool = Pool(32)
    ram_pressure_axis = []
    jet_tracer_axis = []
    for ram_pressure_axis_1, jet_tracer_axis_1 in pool.map(x_loop, np.arange(len(x_axis)), chunksize=1):
        jet_tracer_axis.append(jet_tracer_axis_1)
        ram_pressure_axis.append(ram_pressure_axis_1)
#     ram_pressure_axis, jet_tracer_axis = pool.map(x_loop,)
    pool.close()
    
    jet_tracer_fall_x_parallel.append(x_axis[np.array(jet_tracer_axis)<5e-2][0])
    ram_pressure_list_parallel.append(ram_pressure_axis)
    time_parallel.append(get_time_from_snap(snap_data) * unit_time_in_megayr - 15)

In [ ]:
time = np.array(time_parallel)

In [ ]:
real_y = np.flip(x_axis)
real_x = time + 15

dx = (real_x[1]-real_x[0])/2.
dy = (real_y[1]-real_y[0])/2.
extent = [real_x[0]-dx, real_x[-1]+dx, real_y[0]-dy, real_y[-1]+dy]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
im = plt.imshow(np.array(ram_pressure_list_parallel).T * rho_to_numdensity, 
                extent=extent, aspect='auto', norm=colors.LogNorm(vmin=5e2, vmax=1e-2))#vmin=1e-9, vmax=1e-5))#
plt.colorbar(im, label=r'number density, cm$^{-3}$')
#plt.colorbar(im, label='ram pressure, Bar')
# ax.set(xticks=np.arange(0, len(time), 10), xticklabels=np.arange(time.min(), time.max(), 10));
plt.gca().invert_yaxis()
plt.plot(real_x, np.array(jet_tracer_fall_x_parallel), c='white', zorder=100)
plt.xlim(15, real_x.max())
# plt.ylim(15, 500)
plt.xlabel('time, Myr')
plt.ylabel('r, pc')

In [ ]:
plt.plot(time, top_ram_pressure_x, label='position of the highest ram pressure bump')
plt.plot(time, jet_tracer_fall_x, label='position of jet tracer grop')
plt.legend()
plt.xlabel('time, Myr')
plt.ylabel('distance, pc')